In [1]:
# Используя XGBOOST предсказать размер чаевых в датасете TIPS

In [2]:
# pip install xgboost

In [3]:
import seaborn as sns
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [4]:
PATH = './tips'
tips = sns.load_dataset(PATH)
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [5]:
days = {'Mon':0, 'Tue':1, 'Wed':2, 'Thur':3, 'Fri':4, 'Sat':5, 'Sun':6}
tips['day'] = tips['day'].map(days)
tips['sex'] = tips['sex'].astype('category')
tips["sex"] = tips["sex"].cat.codes
tips['smoker'] = tips['smoker'].astype('category')
tips["smoker"] = tips["smoker"].cat.codes
tips['time'] = tips['time'].astype('category')
tips["time"] = tips["time"].cat.codes

tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0,0,6,0,2
1,10.34,1.66,1,0,6,0,3
2,21.01,3.50,1,0,6,0,3
3,23.68,3.31,1,0,6,0,2
4,24.59,3.61,0,0,6,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,1,0,5,0,3
240,27.18,2.00,0,1,5,0,2
241,22.67,2.00,1,1,5,0,2
242,17.82,1.75,1,0,5,0,2


In [6]:
model_data = tips[['tip', 'total_bill','time', 'sex', 'smoker', 'size', 'day']]
X = model_data.iloc[:,1:len(model_data) -1]
Y = model_data.iloc[:,0:1]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
xgbr = xgb.XGBRegressor( 
    base_score = 0.5,
    n_estimators=100,
    max_depth=6 
)
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [8]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgbr, X_train, y_train,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())

Mean cross-validation score: -0.08


In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

y_pred = xgbr.predict(X_test)
print('score: '+ str(xgbr.score(X_test , y_test)))
print('squered error: '+ str(mean_squared_error(y_test, y_pred)))

score: 0.2985742959664166
squered error: 1.5889778572344595
